<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header">
      <a class="navbar-brand" href="0_Dataoving3.ipynb">Dataøving 3</a>
    </div>
    <ul class="nav navbar-nav">
        <li><a href="Oppgave1.ipynb">Oppgave 1 - Introduksjon til spektral lekkasje</a></li>
        <li class="active"><a href="Oppgave2.ipynb">Oppgave 2 - Zero Padding og DTFT</a></li>
        <li><a href="Oppgave3.ipynb">Oppgave 3 - Vindusfunksjoner</a></li>
        <li><a href="Oppgave4.ipynb">Oppgave 4 - Spektrogram: <i>Tilstandsovervåking av maskineri del 2</i></a></li>
    </ul>
  </div>
</nav>

## Zero Padding

#### Tema:
* Oppløsningsbåndbredde
* Zero Padding
* Frekvensdeteksjon

#### Læringsmål:
* Bruke zero padding til å produsere ønsket detaljnivå ved frekvensanalyse.

#### Bibliotek og notebook-konfigurasjon:

In [1]:
from numpy import sin, cos, pi, exp, empty, mean, absolute, angle # Sentrale matematiske funksjoner
from numpy.fft import fft, ifft             # DFT og IDFT
import numpy as np                          # Importer funksjonalitet fra numpy biblioteket med prefiks "np"
import matplotlib.pyplot as plt             # Importer pyplot modulen i matplotlib med prefiks "plt"
from scipy.io import loadmat                # Lesing av data fra '.mat' fil

%matplotlib ipympl

### Introduksjon:

To begrep som dukker opp mye i frekvensanalyse er *vinduslengde* og *oppløsningsbåndbredde*. Disse er nært knyttet til hverandre og kan oppsummeres som følger:

#### Vinduslengde:
* Antallet sampler som brukes til å utregne Diskret Fouriertransformasjon.
* Bokstaven $N$ brukes ofte i denne sammenheng, og kan løst tolkes som "et heltall".

#### Oppløsningsbåndbredde $\Delta f\ / \ \Delta \hat{\omega}$:
* Båndbredden til frekvensområdet i signalet (i $Hz$ eller $rad/sample$), som representeres med ett enkelt element i DFT-sekvensen $X[k]$.
$$ \Delta f = \frac{f_s}{N}$$
$$ \Delta \hat{\omega} = \frac{2\pi}{N}$$
* Den utregnede verdien $X[k]$ for et gitt heltall $k$ "dekker" altså frekvensområdet $k\cdot \Delta f \pm \frac{\Delta f}{2}$.

Dersom det er ønskelig med et høyt detaljnivå for frekvensanalysen av et signal, krever dette altså å ta i bruk en stor vinduslengde $N$. Problemet med å bruke en stor vinduslengde $N$ er at det innebærer å observere signalet over et lengre tidsintervall, og kan føre til en lite informativ estimasjon av frekvensinnnhold dersom dersom frekvensinnholdet til signalet endrer seg mye i løpet av dette tidsintervallet.

Én metode for å øke detaljnivå uten bruke et større antall signalsampler er med ***Zero Padding***. Prinsippet er ganske enkelt å forlenge signalutklippet med et valgfritt antall sampler med verdi lik null, og så utføre DFT av det forlengede signalet. 

![](Figurer/zero_padding_1.png) 

Det er viktig å bemerke at dette ikke gir samme resultat som å øke vinduslengden $N$. Større vinduslengde $N$ betyr at man innhenter mer informasjon imens zero padding heller "fyller ut" med informasjonen som allerede er tilgjengelig. Resultatet blir flere sampler av frekvensinnholdet, men disse vil sample verdier langs det opprinnelige signalutdragets Diskret-Tids Fourier Transformasjon (DTFT) som er et *kontinuerlig* funksjonsuttrykk. Dette er illustrert i de to figurene nedenfor, hvor det er tydelig at signalets DTFT (den røde kurven) forblir uendret, samtidig som zero padding gir flere sampler *langs* den røde kurven.

![](Figurer/zero_padding_2.png) 

`fft()`-funksjonen til `numpy` har en mulighet for å ta i bruk zero padding ved utregning av en DFT-sekvens. Funksonen har en valgfri parameter `n` som angir lengden til det utvidede signalutklippet etter zero padding. 

```python
# Eksempel:
np.fft.fft(xn, n=128)
```

## Oppgave:

Filen `Datafiler/Oppgave2.mat` inneholder 25 sampler av en sinussekvens $x[n]$ merket med nøkkelordet `'x[n]'`, der signalet er samplet med samplingsfrekvensen angitt av verdien merket med nøkkelordet `'fs'`. Kodecellen leser dataene fra `.mat`-filen, og lagrer de i variablene `x_n` og `fs`.

In [2]:
data = loadmat('Datafiler/Oppgave2.mat', squeeze_me=True)
x_n = data['x[n]']
fs = data['fs']

### a)
Bruk zero padding til å regne ut frekvensinnholdet til $x_2[n]$ for 50 punkter i frekvensplanet. Vis resultatet i en figur, og skalér x-aksen slik at den viser fysisk frekvens (Hz).

In [3]:
### BEGIN SOLUTION
nfft=fs*1000
Xk = fft(x_n, n=nfft)
f = np.linspace(0, fs, nfft, endpoint=False)
plt.close(1); plt.figure(1)
plt.plot(f, np.abs(Xk))

f_sine = f[np.argmax(np.abs(Xk))]
print("Sinusbølgens frekvens:",f_sine)
### END SOLUTION

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Sinusbølgens frekvens: 3.458


### b)
Du skal nå finne hva frekvensen til sinssekvensen $x[n]$ er (i Hz). Dette kan gjøres ved å øke oppløsningen, og identifisere hvor toppunktet til Diskret-Tids Fouriertransformen er. Svaret skal oppgis med én desimal presisjon, noe som tilsvarer en oppløsningsbåndbredde $\Delta f = 0.1Hz$. Jobb videre med kodecellen fra oppgave **a** for å løse oppgaven.

*P.S. For store datamengder er det mer hensiktsmessig å lage en kontinuerlig graf med `plot()` istedenfor å lage et spolpediagram med `stem()`.*

For å få en oppløsningsbåndbredde $\Delta f = 0.1Hz$, må vi ha en et tilstrekkelig stort antall datapunkt i DFT-sekvensen. 
\begin{align}
\Delta f &= 0.1 = \frac{f_s}{\text{nfft}} \\
\text{nfft} &= \frac{f_s}{0.1} = 10\cdot f_s 
\end{align}
Vi kan da bruke `np.argmax()` til å finne toppunktet. Kurven har første toppunkt i $f=3.5Hz$, som dermed vil være frekvensen til sinusbølgen.